In [18]:
from google.cloud import bigquery
from google.cloud.bigquery import job
import pandas as pd
import os
from tqdm import tqdm
import pandas as pd
PROJCECT = 'ballosodeuk'
bq = bigquery.Client(project=PROJCECT)
import statsmodels.api as sm 
import numpy as np

In [142]:
# 클라이언트 설정
client = bigquery.Client()
query_name = "1017_쿠폰구매데이터"
# SQL 파일 읽기
# file = './query/쿠팡0403_0509.sql'
file = f'../query/{query_name}.sql'
with open(file, 'r') as file:
    query = file.read()


In [143]:
job_config = bigquery.QueryJobConfig()
query_job = client.query(query, job_config=job_config)

In [144]:
df = query_job.to_dataframe()

c:\Users\Owner\miniconda3\envs\pymc\lib\site-packages\google\cloud\bigquery\table.py:2309: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
c:\Users\Owner\miniconda3\envs\pymc\lib\site-packages\google\cloud\bigquery\table.py:2323: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
c:\Users\Owner\miniconda3\envs\pymc\lib\site-packages\google\cloud\bigquery\table.py:2337: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)


In [145]:
df.to_parquet('./0503_1021_earningSpending.parquet')

In [147]:
df.rename(columns={'9000-12999':'9000-10999'},inplace=True)
df.entropy.mean()


0.34740111715437655

In [148]:
df.columns

Index(['User_ID', 'Mean_Spend_Ratio', 'entropy', '0-1000', '1000-2999',
       '3000-4999', '5000-6999', '7000-8999', '9000-10999', '11000 - 12999',
       '13000 - 14999', '15000 - 16999', '17000 - 18999', '19000 - 20999',
       '21000 - 22999', '23000 - 24999', '25000 - 26999', '27000 - 28999',
       '29000 - 30999', '31000 - 32999', '33000 - 34999', '35000 - 36999',
       '37000 - 38999', '39000 - 40999', '41000 -'],
      dtype='object')

- 통계치

In [149]:
def ci_stats(df,col):
    mean = df[col].mean()
    std = df[col].std()
    n = len(df)
    ci_upper = mean + 1.96 * std / np.sqrt(n)
    ci_lower = mean - 1.96 * std / np.sqrt(n)
    
    return round(mean,2), round(ci_upper,2), round(ci_lower,2)

spend_ratio = ci_stats(df,'Mean_Spend_Ratio')
entropy = ci_stats(df,'entropy')

print(spend_ratio, entropy)

(0.68, 0.68, 0.68) (0.35, 0.35, 0.34)


In [150]:
from sklearn.preprocessing import MinMaxScaler

sc = MinMaxScaler()
tst = df[['Mean_Spend_Ratio','entropy']]

tst_scaled = sc.fit_transform(tst)
pd.DataFrame(tst_scaled).corr()

,0,1
0,1.000000,-0.072604
1,-0.072604,1.000000


In [151]:
df

,User_ID,Mean_Spend_Ratio,entropy,0-1000,1000-2999,3000-4999,5000-6999,7000-8999,9000-10999,11000 - 12999,...,23000 - 24999,25000 - 26999,27000 - 28999,29000 - 30999,31000 - 32999,33000 - 34999,35000 - 36999,37000 - 38999,39000 - 40999,41000 -
0,2e4fd5ca-3ff7-49ba-ba0e-b735148a76fe,0.434666,0.00,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,bfc8d466-1f9e-4809-9214-fac474c05bc8,0.601046,1.00,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,053149c5-f809-4192-a8a7-975718793ada,0.852169,0.00,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,500fc07f-4311-414f-b7ad-0c39245d7aaa,0.615150,1.52,0,0,1,2,0,2,0,...,0,0,0,0,0,0,0,0,0,0
4,398482d7-d029-4284-8423-f2e38d8a3fc4,0.757576,0.00,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163446,e3b72643-c424-4d38-b0d0-adc4dbe827fd,0.901113,0.00,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
163447,7acc3820-0aab-4769-9344-77eabe434a11,0.918086,0.00,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
163448,97c6c099-5452-4ff6-bcc9-dc342af38efa,1.000000,0.00,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
163449,d121650d-6909-435b-9990-40ce5b084421,0.910262,0.00,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 소비 강도의 증가에 따른 구매금액 범주 속할 확률 증가분

#### 1만원 이하 구매자의 경우, 탕진 확률이 낮다. 이상의 경우, 탕진 확률이 높다


#### 9월 소량 데이터
- print(np.exp(low_coef)) #1.1만원 이하 -> 강도 증가 시 0.14 배 낮아짐
- print(np.exp(low_mid)) #2.1만원 이하 -> 강도 증가 시 2.15배 높아짐
- print(np.exp(mid)) #3.1만원 이하 -> 강도 증가 시 12배 높아짐
- print(np.exp(high)) #4.1 만 이하, 이상 -> 강도 증가 시 30배 높아짐

#### 5~10월 전체 (16만명)
- 1.1만원 이하 : 강도 증가 시 0.04배 낮아짐
- 2.1만원 이하 : 강도 증가 시 1.78배 높아짐
- 3.1만원 이하 : 강도 증가 시 3.88배 높아짐
- 4.1만원 이하 : 강도 증가 시 15.4배 높아짐

>1. 따라서, 1.1만원 이하 구매의 경우 한탕 소진이 아니다. <br>
>2. 이 이상 높아질 시, 한탕 소진으로 모든 금액을 소진할 확률이 높다.
>3. 큰 쿠폰 구매자는 돈을 모아서 도달하고, 소액 쿠폰 구매는 잔액을 거의 대부분 남겨놓는 경향이 있다.
>4. 이 금액이 쇼핑지원금 타겟이다

In [152]:
def pre_prep(df):
    df['low'] = df[['0-1000', '1000-2999', '3000-4999', '5000-6999','7000-8999', '9000-10999', ]].sum(axis=1)
    df['low-mid'] = df[['11000 - 12999', '13000 - 14999', '15000 - 16999', '17000 - 18999', '19000 - 20999']].sum(axis=1)
    df['mid'] = df[['21000 - 22999', '23000 - 24999', '25000 - 26999', '27000 - 28999','29000 - 30999']].sum(axis=1)
    df['high'] = df[['31000 - 32999', '33000 - 34999', '35000 - 36999', '37000 - 38999', '39000 - 40999', '41000 -']].sum(axis=1)

    df_ = df[['User_ID','Mean_Spend_Ratio','low','low-mid','mid','high']].copy()
    df_ = df_[df_['Mean_Spend_Ratio'].notna()]
    df_ = df_.dropna(subset=['Mean_Spend_Ratio'])  # 결측값 제거
    df_ = df_[~df_['Mean_Spend_Ratio'].isin([float('inf'), -float('inf')])] 

    return df_


In [153]:
def logit_func(df_):
    X = df_['Mean_Spend_Ratio']
    y = df_.iloc[:,2:]
    y['low'] = y['low'].apply(lambda x : 1 if x > 1 else x)
    y['low-mid'] = y['low-mid'].apply(lambda x : 1 if x > 1 else x)
    y['mid'] = y['mid'].apply(lambda x : 1 if x > 1 else x)
    y['high'] = y['high'].apply(lambda x : 1 if x > 1 else x)

    results = {}
    for column in y.columns:
        y_column = y[column]
        X_with_const = sm.add_constant(X)
        model = sm.Logit(y_column, X_with_const).fit()
        results[column] = model.summary()

    return results


In [154]:
import pandas as pd
import numpy as np

def get_results(results):
    extracted_results = {}
    for column, summary in results.items():
        # summary.tables[1]에서 계수와 p-value 추출
        # 첫 번째 행은 상수항이므로, 두 번째 행부터 시작
        for i in range(1, len(summary.tables[1].data)):  # 첫 번째 행(상수항)을 제외하고 반복
            coef = summary.tables[1].data[i][1]  # 계수
            p_value = summary.tables[1].data[i][4]  # p-value
            variable_name = summary.tables[1].data[i][0]  # 변수 이름
            extracted_results[f"{column}_{variable_name}"] = {'coef': coef, 'p-value': p_value}        

    # DataFrame 생성
    df = pd.DataFrame(extracted_results).T

    # odds-ratio 계산
    def safe_exp(x):
        try:
            # x를 float로 변환
            float_x = float(x)
            return np.exp(float_x)  # exp 적용
        except (ValueError, TypeError):
            return np.nan  # 변환 불가능한 경우 NaN 반환

    df['odds-ratio'] = df['coef'].apply(safe_exp)
        
    return df

In [141]:
df_result = get_results(logit_func(pre_prep(df)))
df_result

Optimization terminated successfully.
         Current function value: 0.484430
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.342526
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.141695
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.269407
         Iterations 8


,coef,p-value,odds-ratio
low_const,2.7560,0.000,15.736770
low_Mean_Spend_Ratio,-1.9041,0.000,0.148957
low-mid_const,-2.6447,0.000,0.071027
low-mid_Mean_Spend_Ratio,0.7709,0.000,2.161711
mid_const,-5.2627,0.000,0.005181
mid_Mean_Spend_Ratio,2.4887,0.000,12.045607
high_const,-5.0262,0.000,0.006564
high_Mean_Spend_Ratio,3.3980,0.000,29.904232


In [155]:
# 전체 5/3 ~ 10/21
df_result = get_results(logit_func(pre_prep(df)))
df_result

Optimization terminated successfully.
         Current function value: 0.467827
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.404297
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.186117
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.385834
         Iterations 7


,coef,p-value,odds-ratio
low_const,3.6969,0.000,40.322112
low_Mean_Spend_Ratio,-3.1458,0.000,0.043032
low-mid_const,-2.2136,0.000,0.109306
low-mid_Mean_Spend_Ratio,0.5776,0.000,1.781757
mid_const,-4.0041,0.000,0.018241
mid_Mean_Spend_Ratio,1.3579,0.000,3.888020
high_const,-3.8508,0.000,0.021263
high_Mean_Spend_Ratio,2.7391,0.000,15.473053


### 엔트로피 구간 별 통계

In [167]:
df

,User_ID,Mean_Spend_Ratio,entropy,0-1000,1000-2999,3000-4999,5000-6999,7000-8999,9000-10999,11000 - 12999,...,31000 - 32999,33000 - 34999,35000 - 36999,37000 - 38999,39000 - 40999,41000 -,low,low-mid,mid,high
0,2e4fd5ca-3ff7-49ba-ba0e-b735148a76fe,0.434666,0.00,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,bfc8d466-1f9e-4809-9214-fac474c05bc8,0.601046,1.00,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,1
2,053149c5-f809-4192-a8a7-975718793ada,0.852169,0.00,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
3,500fc07f-4311-414f-b7ad-0c39245d7aaa,0.615150,1.52,0,0,1,2,0,2,0,...,0,0,0,0,0,0,5,0,0,0
4,398482d7-d029-4284-8423-f2e38d8a3fc4,0.757576,0.00,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163446,e3b72643-c424-4d38-b0d0-adc4dbe827fd,0.901113,0.00,0,0,0,0,0,2,0,...,0,0,0,0,0,0,2,0,0,0
163447,7acc3820-0aab-4769-9344-77eabe434a11,0.918086,0.00,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
163448,97c6c099-5452-4ff6-bcc9-dc342af38efa,1.000000,0.00,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
163449,d121650d-6909-435b-9990-40ce5b084421,0.910262,0.00,0,3,0,0,0,0,0,...,0,0,0,0,0,0,3,0,0,0


In [163]:
df.entropy.min(), df.entropy.max()

(0.0, 3.23)

In [171]:
import pandas as pd
import numpy as np

# entropy 구간 설정
bins = [0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0, 2.2, 2.4, 2.6, 2.8, 3.0, 3.2, 3.4]  # 구간 설정
labels = ['0-0.2', '0.2-0.4', '0.4-0.6', '0.6-0.8', '0.8-1.0', '1.0-1.2', 
          '1.2-1.4', '1.4-1.6', '1.6-1.8', '1.8-2.0', '2.0-2.2', '2.2-2.4',
          '2.4-2.6','2.6-2.8','2.8-3.0','3.0-3.2','3.2-3.4']  # 구간 레이블
df['entropy_bins'] = pd.cut(df['entropy'], bins=bins, labels=labels, right=False)

# 각 구간별 유저 수와 구매 횟수 집계
result = df.groupby('entropy_bins').agg(
    user_count=('User_ID', 'nunique'),  # 유저 수
    Mean_Spend_Ratio=('Mean_Spend_Ratio','mean'),
    low=('low', 'sum'),  # 총 구매 횟수
    low_mid=('low-mid', 'sum'),  # 총 구매 횟수
    mid=('mid', 'sum'),  # 총 구매 횟수
    high=('high', 'sum'),  # 총 구매 횟수
).reset_index()

# 결과 출력
print(result)

   entropy_bins  user_count  Mean_Spend_Ratio     low  low_mid   mid   high
0         0-0.2      111514          0.694517  144995    11489  4470  18119
1       0.2-0.4         134          0.553584    2203        5     1     23
2       0.4-0.6        1103          0.579893    9698       54    10     28
3       0.6-0.8        2255          0.602560   13324      243    35     77
4       0.8-1.0       14566          0.645597   50986     3047   509    979
5       1.0-1.2       21030          0.681010   37348     6193  1628   3267
6       1.2-1.4        1774          0.582428   10705      408    67    113
7       1.4-1.6        8720          0.639128   29174     3711   678    999
8       1.6-1.8         376          0.532025    2721      143    25     41
9       1.8-2.0         998          0.576698    5308      537    72     96
10      2.0-2.2         717          0.618378    2609      476   106    135
11      2.2-2.4         221          0.540285    1145      211    39     40
12      2.4-

C:\Users\Owner\AppData\Local\Temp\ipykernel_39696\2108345262.py:12: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  result = df.groupby('entropy_bins').agg(


In [172]:
pd.DataFrame(result)

,entropy_bins,user_count,Mean_Spend_Ratio,low,low_mid,mid,high
0,0-0.2,111514,0.694517,144995,11489,4470,18119
1,0.2-0.4,134,0.553584,2203,5,1,23
2,0.4-0.6,1103,0.579893,9698,54,10,28
3,0.6-0.8,2255,0.602560,13324,243,35,77
4,0.8-1.0,14566,0.645597,50986,3047,509,979
5,1.0-1.2,21030,0.681010,37348,6193,1628,3267
6,1.2-1.4,1774,0.582428,10705,408,67,113
7,1.4-1.6,8720,0.639128,29174,3711,678,999
8,1.6-1.8,376,0.532025,2721,143,25,41
9,1.8-2.0,998,0.576698,5308,537,72,96
